1- Implementar uma função para determinar a classe de estabilidade de Pasquil para diferentes condições atmosféricas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Pegando o caminho do diretório que estou
rootPath = os.getcwd()
print(rootPath)

# Definindo o diretório com os arquivos metar
metSuperficiePath = rootPath +'\\inputs\\meteoSuperficie'
print(metSuperficiePath)

ezAermetPath = rootPath + '\\inputs\\ezAermet'
print(ezAermetPath) 

#Listando os arquivos dentro do diretório
files = os.listdir(ezAermetPath)
print(files)]

In [ ]:
# Leitura dos dados
df = pd.read_csv('C:\\Users\\anaju\\OneDrive\\Documentos\\GitHub\\Controle_AnaPietroski\\inputs\\ezAermet\\METEO.csv')
df

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Leitura dos dados
df = pd.read_csv('C:\\Users\\anaju\\OneDrive\\Documentos\\GitHub\\Controle_AnaPietroski\\inputs\\ezAermet\\METEO.csv')

# Correção do ano
df['year'] = df['year'].apply(lambda x: 2000 + x if x < 100 else x)

# Criação da coluna datetime
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])

# Definição do índice datetime
df.set_index('datetime', inplace=True)

# Seleção de 5 dias no verão e 5 dias no inverno
dias_verao = df[(df.index.month >= 12) | (df.index.month <= 2)].sample(n=5, random_state=1)
dias_inverno = df[(df.index.month >= 6) & (df.index.month <= 8)].sample(n=5, random_state=1)

# Combinação dos dias selecionados
dias_selecionados = pd.concat([dias_verao, dias_inverno])

# Filtrar apenas colunas numéricas
colunas_numericas = dias_selecionados.select_dtypes(include=[np.number]).columns

# Cálculo da média horária dos parâmetros numéricos
media_horaria = dias_selecionados[colunas_numericas].groupby(dias_selecionados.index.date).mean()

# Função para determinar a classe de estabilidade atmosférica
def definir_classe_estabilidade(linha):
    cobertura_nuvens = linha['cloud cover ']  # Ajuste conforme a saída do print(df.columns)
    velocidade_vento = linha['wind speed ']      # Ajuste conforme a saída do print(df.columns)
    fluxo_calor_sensivel = linha['sensible']  # Ajuste conforme a saída do print(df.columns)
    
    if fluxo_calor_sensivel > 10:
        insolacao = 'forte'
    elif 3 < fluxo_calor_sensivel <= 10:
        insolacao = 'moderada'
    else:
        insolacao = 'fraca'

    if cobertura_nuvens < 3:
        if velocidade_vento < 2:
            return 'A' if insolacao == 'forte' else 'B'
        elif 2 <= velocidade_vento < 3:
            return 'B' if insolacao == 'forte' else 'C'
        else:
            return 'C'
    elif 3 <= cobertura_nuvens < 7:
        if velocidade_vento < 2:
            return 'B' if insolacao == 'forte' else 'C'
        elif 2 <= velocidade_vento < 5:
            return 'C'
        else:
            return 'D'
    else:
        if velocidade_vento < 2:
            return 'C'
        elif 2 <= velocidade_vento < 5:
            return 'D'
        else:
            return 'E'

# Aplicação da função para determinar a classe de estabilidade
media_horaria['classe_estabilidade'] = media_horaria.apply(definir_classe_estabilidade, axis=1)

# Exibição dos resultados
print(media_horaria[['cloud cover ', 'wind speed ', 'sensible', 'classe_estabilidade']])


In [ ]:
# Função para calcular sigma_y e sigma_z
def sigmaYZ(x, classe, urbOrRural):
    if urbOrRural == 'urbano':
        if classe == 'A' or classe == 'B':
            sigmaY = 0.32 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.24 * x * (1 + 0.001 * x) ** 0.5
        elif classe == 'C':
            sigmaY = 0.22 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.20 * x
        elif classe == 'D':
            sigmaY = 0.16 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.14 * x * (1 + 0.0003 * x) ** (-0.5)
        elif classe == 'E' or classe == 'F':
            sigmaY = 0.11 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.08 * x * (1 + 0.0015 * x) ** (-0.5)
        else:
            raise ValueError('Classe de estabilidade errada')
    elif urbOrRural == 'rural':
        if classe == 'A':
            sigmaY = 0.22 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.20 * x
        elif classe == 'B':
            sigmaY = 0.16 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.12 * x
        elif classe == 'C':
            sigmaY = 0.11 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.08 * x * (1 + 0.0002 * x) ** (-0.5)
        elif classe == 'D':
            sigmaY = 0.08 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.06 * x * (1 + 0.0003 * x) ** (-0.5)
        elif classe == 'E':
            sigmaY = 0.06 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.03 * x * (1 + 0.0003 * x) ** (-1)
        elif classe == 'F':
            sigmaY = 0.04 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.016 * x * (1 + 0.0003 * x) ** (-1)
        else:
            raise ValueError('Classe de estabilidade errada')
    else:
        raise ValueError('Tipo de ambiente deve ser "urbano" ou "rural"')
    
    return sigmaY, sigmaZ

In [ ]:
def sobrelevacaoBriggs(Fb, Vs, d, u, deltaT, Ts, He, tipo_atmosfera):
    """
    Calcula a sobrelevação da pluma usando o modelo de Briggs.
    """
    # Determinar se a atmosfera é instável ou estável com base na classe de Pasquill
    tipo_atmosfera = 'instavel' if classe in ['A', 'B', 'C', 'D'] else 'estavel'
    
    if tipo_atmosfera == 'instavel':
        if Fb < 55:
            deltaTe = 0.0297 * Ts * (Vs / (d ** (2/3)))
        else:
            deltaTe = 0.00575 * Ts * (Vs ** (2/3) / (d ** (1/3)))
    else:
        deltaTe = 0.01958 * Ts * (Vs / (d ** (1/3))) * (Fb ** 0.5)

    if tipo_atmosfera == 'instavel':
        if Fb < 55:
            if deltaT < deltaTe:
                H = He + 3 * d * (Vs / u)
            else:
                H = He + 21.425 * (Fb ** (3/4) / (u ** (1/2)))
        else:
            if deltaT < deltaTe:
                H = He + 3 * d * (Vs / u)
            else:
                H = He + 2.6 * (Fb ** (1/3) / (Ts ** (1/3)))
    else:
        if deltaT < deltaTe:
            H = He + 1.5 * (Fb ** (1/3) / (u ** (1/3)))
        else:
            H = He + 2.6 * (Fb ** (1/3) / (Ts ** (1/3)))

    return H - He

In [ ]:
def modeloGaussiano(qs,sigmaY,sigmaZ,u,y,z,H):
    termo1 = qs/(2*np.pi*sigmaY*sigmaZ*u)
    termo2 = np.exp((-y**2)/(2*sigmaY**2))
    termo3 = np.exp((-(z-H)**2)/(2*sigmaZ**2)) + np.exp((-(z+H)**2)/(2*sigmaZ**2))
    conc = termo1*termo2*termo3
    conc = conc*10**6
    return conc

In [ ]:
# Utilizando os dados do trabalho 1


In [ ]:
#lendo os dados da tabela da AP42
